In [ ]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import os
import shutil
import zipfile

In [ ]:
# To connect your drive for storing the data later on (can also be downloaded from colab without storing in drive)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from skimage import exposure
from skimage.color import rgb2gray
import cv2
import math
from tqdm import tqdm

In [ ]:
# The cross-validation data can be downloaded from
# https://www.kaggle.com/datasets/nih-chest-xrays/data
# Set the downloaded zip file's name:

cross_val_zip_name = ''


# Unzipping the files to a directory
cross_val_zip = cross_val_zip_name + '.zip'
os.mkdir('Cross_Validation')
with zipfile.ZipFile(cross_val_zip,"r") as zip_ref:
    zip_ref.extractall('Cross_Validation')

In [ ]:
# Creating the output directory for each class:
os.mkdir("sample_data")
for i in range(3,18):
    dir_str = 'sample_data/' + str(i*5) + '-0'
    os.mkdir(dir_str)
    dir_str = 'sample_data/' + str(i*5) + '-1'
    os.mkdir(dir_str)

In [ ]:
# Sampling 30000 images for patients between 15 and 89:
labels = pd.read_csv('Cross_Validation/Data_Entry_2017.csv')  # the csv file can be downloaded from https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760887468
labels = labels[labels['Patient Age'] > 14]
labels = labels[labels['Patient Age'] < 90]
labels_sample = labels.sample(n = 30000, random_state= 42)
img_indexes = labels_sample['Image Index'].to_numpy()

In [ ]:
# Copying the sampled images into the directory of their class:
for i in range(12):
    if i < 9:
        current_dir = 'Cross_Validation/images_00' + str(i+1) + '/images'
    else:
        current_dir = 'Cross_Validation/images_0' + str(i+1) + '/images'
    for img in os.listdir(current_dir):
        if str(img) in img_indexes:
            current_index = int(np.where(labels_sample['Image Index']==str(img))[0])
            if labels_sample.iloc[current_index, 5] == 'M':
                gender = '1'
            else:
                gender = '0'
            age = str(5 * round(labels_sample.iloc[current_index, 4] / 5))
            dir_name = 'sample_data/' + age + '-' + gender
            img_path = current_dir + '/' + str(img)
            shutil.copy(img_path, dir_name)
os.remove('sample_data/90-0')
os.remove('sample_data/90-1')

In [ ]:
# Creating a zip for the sampled data:
shutil.make_archive('sample_data', 'zip', 'sample_data')

'c:\\CBS\\Machine Learning\\Final\\CROSS VAL\\sample_data.zip'

# Preprocessing

In [ ]:
# The preprocessing function is the same as it was for the earlier preprocessing, except without image flipping:

def preprocessing_function(edge_enhance, equalization, blur, image):

    # apply gaussian blur to the image
    image = cv2.GaussianBlur(image, (blur, blur), 0 )
    # edge enhance using PIL
    if edge_enhance:
        image = Image.fromarray(image)
        image = image.filter(ImageFilter.EDGE_ENHANCE)
        image = np.array(image)
    else:
        image = image
    
    # equalization
    if equalization == 'histogram':
        image = cv2.equalizeHist(image)
    elif equalization == 'adaptive':
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        image = clahe.apply(image)
    else: 
        image = image
    # rotate
    image = image

    return image

In [ ]:
# The image cropping is the same as it was for the earlier preprocessing:

def crop_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # threshold 
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    hh, ww = thresh.shape

    # make bottom 2 rows black where they are white the full width of the image
    thresh[hh-3:hh, 0:ww] = 0

    # get bounds of white pixels
    white = np.where(thresh==255)
    xmin, ymin, xmax, ymax = np.min(white[1]), np.min(white[0]), np.max(white[1]), np.max(white[0])
    hd = ymax-ymin
    wd = xmax-xmin
    g = 0 
    j = 0
    m = 0
    f = 0
    if hd > wd and (ymin <2 or ymax + 2 > img.shape[0]):
        g = 0
        j = 0
        t = (hd-wd) / 2 
        if xmin - math.floor(t) >= 0 and xmax + math.ceil(t) <= img.shape[1]:
            m = math.floor(t)
            f = math.ceil(t)
        elif xmin - math.floor(t) >= 0:
            f = img.shape[1]-xmax
            m = ((2*t)-f)
        else:
            m = xmin
            f = ((2*t)-m)
    elif hd > wd and (ymin >= 2 or ymax + 2 <= img.shape[0]):
        g = 2
        j = 2
        t = ((hd+4)-wd) / 2
        if xmin - math.floor(t) >= 0 and xmax + math.ceil(t) <= img.shape[1]:
            m = math.floor(t)
            f = math.ceil(t)
        elif xmin - math.floor(t) >= 0:
            f = img.shape[1]-xmax
            m = ((2*t)-f)
        else:
            m = xmin
            f = ((2*t)-m)
    elif wd > hd and (xmin < 2 or xmax +2 > img.shape[1]):
        m = 0 
        f = 0
        t = (wd-hd) / 2 
        if ymin - math.floor(t) >= 0 and ymax + math.ceil(t) <= img.shape[0]:
            g = math.floor(t)
            j = math.ceil(t)
        elif ymin - math.floor(t) >= 0:
            j = img.shape[0]-ymax
            g = ((2*t)-j)
        else:
            g = ymin
            j = ((2*t)-g)
    elif wd > hd and (xmin >= 2 or xmax + 2 <= img.shape[1]):
        m = 2 
        f = 2
        t = ((wd+4)-hd) / 2 
        if ymin - math.floor(t) >= 0 and ymax + math.ceil(t) <= img.shape[0]:
            g = math.floor(t)
            j = math.ceil(t)
        elif ymin - math.floor(t) >= 0:
            j = img.shape[0]-ymax
            g = ((2*t)-j)
        else:
            g = ymin
            j = ((2*t)-g)
    elif wd == hd:
      if wd + 4 <= img.shape[1] and hd + 4 <= img.shape[0]:
        g = 2 
        j = 2
        m = 2
        f = 2
      else: 
        g = 0 
        j = 0
        m = 0
        f = 0
    # crop the image at the bounds adding back the two blackened rows at the bottom
    if hd + 4 > img.shape[1] or wd + 4 > img.shape[0]:
        crop = img[:,:,0]
    elif hd == 0 or wd == 0:
        crop = img[:,:,0]
    else:
        xx = int(ymin-g)
        xy = int(ymax+j)
        yx = int(xmin-m)
        yy = int(xmax+f)
        crop = img[xx:xy, yx:yy,0]
    if crop.shape[0] != crop.shape[1]:
      crop = img[:,:,0]
    return crop

In [ ]:
# Making a directory for the output:
os.mkdir('/cross_val_content/aug/')
i = 15
while i < 90:
  j = 0
  while j < 2:
    os.mkdir('/cross_val_content/aug/'+ str(i) + "-" + str(j))
    j += 1
  i += 5

In [ ]:
# Preprocessing the images:
data_dir = '/cross_val_content/cross_val_data/' # path to the previously created folder directory
target_path = '/cross_val_content/aug/' # Create the augmented folder in the same preluding path
# Using the same preprocessing arguments as for the earlier preprocessing:
preprocessing_args = {
    'edge_enhance': False,
    'equalization': 'histogram',
    'blur': 3
}
errors = []
shape = []
for folder in os.listdir(data_dir):
    # add folder to the data_dir path
    folder_path = data_dir + folder + '/'
    print(folder_path)
    for image in os.listdir(folder_path):
        try:
            img = cv2.imread(folder_path + image, cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            # cropping 
            print(folder_path + image)
            img = crop_image(img)
            # check that the image has no 0 in its shape, if so. reload it
            if 0 in img.shape:
                img = cv2.imread(folder_path + image, cv2.IMREAD_GRAYSCALE)
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                img = img[:,:,0]
            
            temp = preprocessing_function(preprocessing_args['edge_enhance'],  
                                          preprocessing_args['equalization'],
                                          preprocessing_args['blur'],
                                          img)

            # add a _i to the image name
            path = target_path + folder + '/' + image[:-4] + '_0.png'
            cv2.imwrite(path, temp)
            shape.append(temp.shape)
        except Exception as e:
            errors.append([e, folder_path + image])
            print("error")
            pass

In [ ]:
!zip -r /content/drive/MyDrive/Cross_validation_preprocessed.zip /cross_val_content/aug/ #change the first path if you have not connected your drive and second to whatever you have set it to earlier